In [100]:
#lee logs de ingestas 

In [101]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from ssl import create_default_context
import requests
from getpass import getpass
import pandas as pd
import numpy as np
import json
from datetime import datetime
import parametros

In [102]:
#FECHA ACTUAL 

In [103]:
now = datetime.now()
ahora = str(now.strftime("%Y-%m-%dT%H:%M:%S"))

In [104]:
ahora

'2021-09-14T20:40:59'

In [105]:
#conecta elasticsearch

In [106]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
)

In [107]:
#Leyendo archivo log sonicwall

In [108]:
 try:
    #logread = pd.read_csv('../archivos/sonicwall-curado.log', sep='\t', encoding = 'utf-8')
    header_list = ["column1"]
    logread = pd.read_csv('../archivos/sonicwall-curado.log', header=None,names=header_list, encoding = 'utf-8')
    logread.fillna('', inplace=True)
   
 except:
    print("fecha:",ahora,"- Error leyendo archivo sonicwall-curado.log")
    exit()

In [109]:
logread

,column1
0,Fecha maxima en sonic curado: 2021-06-01 10:50...
1,Fecha maxima en indice crudo: 2021-06-21 23:33:00
2,Fecha: 2021-06-21 18:33:02.616155 - Datos CAT...
3,Fecha: 2021-06-21 18:33:02.616155 - Datos DOM...
4,Fecha: 2021-06-21 18:33:02.616155 - Insertado...
...,...
39244,control fr
39245,control 2
39246,control 3
39247,control 4


In [110]:
list(logread.columns)

['column1']

In [111]:
indice = parametros.monitor_index

In [112]:
use_these_keys = ['column1', '@timestamp']

In [113]:
#Insertando datos en indice

In [114]:



def filterKeys(document):
   return {key: document[key] for key in use_these_keys }
timestamp = datetime.now()
logread['@timestamp'] = timestamp.isoformat()



def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": index,
                "_source": filterKeys(document),
            }
salida = helpers.bulk(es, doc_generator(logread))
print("fecha:",ahora,"- Insertados en sonicmonitor:", salida[0])

fecha: 2021-09-14T20:40:59 - Insertados en sonicmonitor: 39249
